In [1]:
!pip install selenium

In [2]:
!pip install webdriver_manager --upgrade

In [1]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


from urllib.parse import urljoin

In [ ]:
# 문과대학

# 기본 URL 설정
base_url = "https://libart.yonsei.ac.kr/libart/board/notice.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        td_tags = row.select('td')  # 모든 <td> 태그 추출

        if title_tag and len(td_tags) > 0:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            date = "20" + td_tags[-1].get_text(strip=True)  # 연도 맞춤 작성일

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '문과대학',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <dd> 태그에서 텍스트를 추출
    context_tag = soup.select_one('dd .fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)
    time.sleep(1)  # 서버 부하 방지를 위한 대기 시간 설정

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "문과대학_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '문과대학_공지사항.csv'로 저장되었습니다.


In [ ]:
# df['link'][0]
# df['context'][0]

'https://libart.yonsei.ac.kr/libart/board/notice.do?mode=view&articleNo=213970&article.offset=0&articleLimit=10'

In [ ]:
# 상경대학

# 기본 URL 설정
base_url = "https://yce.yonsei.ac.kr/ybe/notice/notice.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        td_tags = row.select('td')  # 모든 <td> 태그 추출

        if title_tag and len(td_tags) > 0:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            date = "20" + td_tags[-1].get_text(strip=True)  # 연도를 맞춘 작성일

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '상경대학',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <dd> 태그에서 텍스트를 추출
    context_tag = soup.select_one('dd .fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)
    time.sleep(1)  # 서버 부하 방지를 위한 대기 시간 설정

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "상경대학_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '상경대학_공지사항.csv'로 저장되었습니다.


In [ ]:
# 경영대학

# 기본 URL 설정
base_url = "https://ysb.yonsei.ac.kr/board.asp"

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(page):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    # 페이지 번호를 포함한 요청 파라미터 설정
    params = {
        'mid': 'm06_01',
        'page': page
    }
    response = requests.get(base_url, params=params)
    
    # 인코딩 문제 해결 (응답의 인코딩을 명시적으로 설정)
    response.encoding = 'euc-kr'  # utf-8로 하면 한글이 깨짐

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a[href*="board.asp?act=view"]')  # 제목 추출
        date_tag = row.select_one('td.board_date')  # 작성일 추출

        if title_tag and date_tag:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            date = date_tag.get_text(strip=True)  # 작성일 추출

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '경영대학',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.encoding = 'euc-kr'  # 인코딩 설정
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 #BoardContent 태그에서 텍스트를 추출
    context_tag = soup.select_one('#BoardContent')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(1, 6):
    print(f"{page}페이지 크롤링 중...")
    scrape_page(page)
    time.sleep(1)  # 서버 부하 방지를 위한 대기 시간 설정

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "경영대학_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '경영대학_공지사항.csv'로 저장되었습니다.


In [ ]:
# 공과대학

# 기본 URL 설정
base_url = "https://engineering.yonsei.ac.kr/engineering/board/notice.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):  # 각 공지사항이 있는 행 선택
        title_tag = row.select_one('a.c-board-title')
        td_tags = row.select('td')  # <td> 태그 모두 선택

        if title_tag and len(td_tags) > 0:  # <td> 태그가 존재하는지 확인
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()  # 공지 태그 제거
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환

            # 작성일은 마지막 <td> 태그에서 추출
            date = "20" + td_tags[-1].get_text(strip=True)  # 연도를 맞춘 작성일

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '공과대학',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)
    time.sleep(1)  # 서버 부하 방지를 위한 대기 시간 설정

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "공과대학_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '공과대학_공지사항.csv'로 저장되었습니다.


In [ ]:
# 이과대학

# 기본 URL 설정
base_url = "http://science.yonsei.ac.kr"
notice_url = f"{base_url}/community/notice"

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(page_number):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    url = f"{notice_url}?p={page_number}"
    response = requests.get(url)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('td.nxb-list-table__title a')
        date_tag = row.select_one('td.nxb-list-table__date')

        if title_tag and date_tag:
            title = title_tag.get_text(strip=True)
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            date = date_tag.get_text(strip=True)

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '이과대학',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="editor-contents">에서 텍스트 추출
    context_tag = soup.select_one('div.editor-contents')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 원하는 페이지 범위 설정 (1페이지부터 5페이지까지 크롤링)
for page in range(1, 6):
    print(f"{page}페이지 크롤링 중...")
    scrape_page(page)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "이과대학_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '이과대학_공지사항.csv'로 저장되었습니다.


In [ ]:
# 생명시스템대학

# 기본 URL 설정
base_url = "https://bio.yonsei.ac.kr/bio/board/notice.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        td_tags = row.select('td')  # 모든 <td> 태그 추출

        if title_tag and len(td_tags) > 0:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            date = "20" + td_tags[-1].get_text(strip=True)  # 연도를 맞춘 작성일

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '생명시스템대학',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('dd > div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "생명시스템대학_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '생명시스템대학_공지사항.csv'로 저장되었습니다.


In [ ]:
# 인공지능융합대학

# 기본 URL 설정
base_url = "https://computing.yonsei.ac.kr/bbs/board.php"
params = {
    'bo_table': 'sub4_4',
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(page):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['page'] = page  # 페이지 번호 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('div.bo_tit a')  # 제목 추출
        date_tag = row.select_one('td.td_datetime')  # 작성일 추출

        if title_tag and date_tag:
            title = title_tag.get_text(strip=True)  # 제목 정제
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            date = date_tag.get_text(strip=True)  # 작성일 추출

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '인공지능융합대학',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div id="bo_v_con">에서 텍스트 추출
    context_tag = soup.select_one('div#bo_v_con')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(1, 6):
    print(f"{page}페이지 크롤링 중...")
    scrape_page(page)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "인공지능융합대학_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '인공지능융합대학_공지사항.csv'로 저장되었습니다.


In [ ]:
# 신과대학

# 기본 URL 설정
base_url = "https://yonshin.yonsei.ac.kr/theology/board02/undergraduate.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        td_tags = row.select('td')  # 모든 <td> 태그 추출

        if title_tag and len(td_tags) > 0:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            date = "20" + td_tags[-1].get_text(strip=True)  # 연도를 맞춘 작성일

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '신과대학',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('dd > div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "신과대학_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '신과대학_공지사항.csv'로 저장되었습니다.


In [ ]:
# 사회과학대학

# 기본 URL 설정
base_url = "https://socsci.yonsei.ac.kr/socsci/community/notice.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        td_tags = row.select('td')  # 모든 <td> 태그 추출

        if title_tag and len(td_tags) > 0:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            date = "20" + td_tags[-1].get_text(strip=True)  # 연도를 맞춘 작성일

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '사회과학대학',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('dd > div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "사회과학대학_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '사회과학대학_공지사항.csv'로 저장되었습니다.


In [77]:
# 음악대학

# 기본 URL 설정
base_url = "https://music.yonsei.ac.kr/music/notice1.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        td_tags = row.select('td')  # 모든 <td> 태그 추출

        if title_tag and len(td_tags) > 0:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            date = "20" + td_tags[-1].get_text(strip=True)  # 연도를 맞춘 작성일

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '음악대학',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('dd > div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "음악대학_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '음악대학_공지사항.csv'로 저장되었습니다.


In [27]:
# 생활과학대학핑

# 기본 URL 설정
base_url = "https://che.yonsei.ac.kr/che/community_che/notice.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        td_tags = row.select('td')  # 모든 <td> 태그 추출

        if title_tag and len(td_tags) > 0:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = base_url + title_tag['href']  # 절대 경로로 변환
            date = "20" + td_tags[-1].get_text(strip=True)  # 연도 맞춤 작성일

            all_notices.append({
                'department': '생활과학대학',
                'title': title,
                'date': date,
                'link': link
            })

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "생활과학대학_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '생활과학대학_공지사항.csv'로 저장되었습니다.


In [79]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

# 기본 URL 설정
base_url = "https://che.yonsei.ac.kr/che/community_che/notice.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        td_tags = row.select('td')  # 모든 <td> 태그 추출

        if title_tag and len(td_tags) > 0:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            date = "20" + td_tags[-1].get_text(strip=True)  # 연도 맞춤 작성일

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '생활과학대학',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('dd > div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "생활과학대학_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '생활과학대학_공지사항.csv'로 저장되었습니다.


In [2]:
# 교육과학대학

# 기본 URL 설정
base_url = "https://ysces.yonsei.ac.kr/yses/edu/notice.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        date_tag = row.select_one('td')  # 작성일 추출

        if title_tag and date_tag:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            date = "20" + date_tag.get_text(strip=True)  # 연도 맞춤 작성일

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '교육과학대학',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('dd > div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "교육과학대학_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '교육과학대학_공지사항.csv'로 저장되었습니다.


In [8]:
# 학부대학

# 기본 URL 설정
base_url = "https://universitycollege.yonsei.ac.kr/fresh/infomation/student.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('td.text-left'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        date_tag = row.select_one('div.c-board-info-m > span:nth-child(2)')  # 작성일 추출

        if title_tag and date_tag:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            date = date_tag.get_text(strip=True)  # 작성일 그대로 가져옴

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '학부대학',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('dd > div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "학부대학_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '학부대학_공지사항.csv'로 저장되었습니다.


In [30]:
# 글로벌인재대학

# 기본 URL 설정
base_url = "https://glc.yonsei.ac.kr/notice/"
params = {
    'mod': 'list',
    'pageid': 1  # 페이지 번호를 설정할 때 사용
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(page_id):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['pageid'] = page_id  # 페이지 번호 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    rows = soup.select('div.kboard-default-cut-strings')
    dates = [tag for tag in soup.select('td.kboard-list-date') if '작성일' not in tag.get_text()]

    for row, date_tag in zip(rows, dates):
        title = row.get_text(strip=True)  # 제목 추출
        link_tag = row.find_parent("a")  # 링크가 포함된 부모 태그 찾기
        link = urljoin(base_url, link_tag['href']) if link_tag else None  # 링크가 있는 경우 절대 경로로 변환
        date = date_tag.get_text(strip=True)  # 작성일 추출

        # 공지사항의 상세 페이지로 이동하여 내용 크롤링
        context = scrape_notice_context(link) if link else "내용 없음"

        all_notices.append({
            'department': '글로벌인재대학',
            'title': title,
            'date': date,
            'link': link,
            'context': context
        })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="content-view">에서 텍스트 추출
    context_tag = soup.select_one('div.content-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(1, 6):  # 페이지 번호는 1부터 시작
    print(f"{page}페이지 크롤링 중...")
    scrape_page(page)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "글로벌인재대학_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '글로벌인재대학_공지사항.csv'로 저장되었습니다.


In [ ]:
# 국어국문학과

# 기본 URL 설정
base_url = "https://koreanlit.yonsei.ac.kr/korean/notice01.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        td_tags = row.select('td')
        
        # `td` 요소가 있고 마지막 `td`가 작성일을 포함하는지 확인
        if title_tag and td_tags and len(td_tags) > 1:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            raw_date = td_tags[-1].get_text(strip=True)  # 마지막 `td` 요소가 작성일
            
            # 날짜에 '20'을 붙여 '2024.10.14' 형식으로 저장
            date = f"20{raw_date}" if raw_date else "날짜 없음"

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '국어국문학과',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('dd > div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "국어국문학과_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '국어국문학과_공지사항.csv'로 저장되었습니다.


In [ ]:
# 중어중문학과

# 기본 URL 설정
base_url = "https://ycll.yonsei.ac.kr/yonseicll/board01.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        date_tags = row.select('td')  # <td> 요소 전체를 선택

        # <td> 요소가 존재하고, 날짜가 포함된 마지막 <td> 요소가 있는지 확인
        if title_tag and date_tags:
            title = title_tag.get_text(strip=True)
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            raw_date = date_tags[-1].get_text(strip=True)
            
            # '24.10.31' 형식을 '2024.10.31'로 변환
            date = f"20{raw_date}" if len(raw_date) == 8 else raw_date

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '중어중문학과',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "중어중문학과_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '중어중문학과_공지사항.csv'로 저장되었습니다.


In [ ]:
# 영어영문학과

# 기본 URL 설정
base_url = "https://ysenglish.yonsei.ac.kr/ysenglish/process/process.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        tds = row.select('td')  # 모든 <td> 요소 선택
        
        # <td> 요소가 있는지 확인하고, 마지막 <td> 요소를 date로 추출
        if title_tag and tds:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            raw_date = tds[-1].get_text(strip=True)  # 작성일 텍스트 추출

            # 날짜가 "YY.MM.DD" 형식인 경우만 추출
            if re.match(r'^\d{2}\.\d{2}\.\d{2}$', raw_date):
                date = f"20{raw_date}"  # "20"을 추가하여 "YYYY.MM.DD" 형식으로 변경
            else:
                continue  # 날짜 형식이 맞지 않으면 스킵

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '영어영문학과',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "영어영문학과_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '영어영문학과_공지사항.csv'로 저장되었습니다.


In [ ]:
# 독어독문학과과

# 기본 URL 설정
base_url = "https://ysgerman.yonsei.ac.kr/ysgerman/ger_notice.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        tds = row.select('td')  # 모든 <td> 요소 선택

        # <td> 요소가 있는지 확인하고, 마지막 <td> 요소를 date로 추출
        if title_tag and tds:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            raw_date = tds[-1].get_text(strip=True)  # 작성일 텍스트 추출

            # 날짜가 "YY.MM.DD" 형식인 경우만 추출
            if re.match(r'^\d{2}\.\d{2}\.\d{2}$', raw_date):
                date = f"20{raw_date}"  # "20"을 추가하여 "YYYY.MM.DD" 형식으로 변경
            else:
                continue  # 날짜 형식이 맞지 않으면 스킵

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '독어독문학과',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "독어독문학과_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '독어독문학과_공지사항.csv'로 저장되었습니다.


In [ ]:
# 불어불문학과

# 기본 URL 설정
base_url = "https://franys.yonsei.ac.kr/franys/notice.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        tds = row.select('td')  # 모든 <td> 요소 선택

        # <td> 요소가 있는지 확인하고, 마지막 <td> 요소를 date로 추출
        if title_tag and tds:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            raw_date = tds[-1].get_text(strip=True)  # 작성일 텍스트 추출

            # 날짜가 "YY.MM.DD" 형식인 경우만 추출
            if re.match(r'^\d{2}\.\d{2}\.\d{2}$', raw_date):
                date = f"20{raw_date}"  # "20"을 추가하여 "YYYY.MM.DD" 형식으로 변경
            else:
                continue  # 날짜 형식이 맞지 않으면 스킵

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '불어불문학과',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "불어불문학과_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '불어불문학과_공지사항.csv'로 저장되었습니다.


In [ ]:
# 노어노문학과

# 기본 URL 설정
base_url = "https://rus.yonsei.ac.kr/russia/borad_main.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        tds = row.select('td')  # 모든 <td> 요소 선택

        # <td> 요소가 있는지 확인하고, 마지막 <td> 요소를 date로 추출
        if title_tag and tds:
            # 제목 텍스트를 불필요한 문자열을 제거하며 추출
            title = title_tag.get_text(strip=True).replace("[공지]", "").replace("[학부]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            raw_date = tds[-1].get_text(strip=True)  # 작성일 텍스트 추출

            # 날짜가 "YY.MM.DD" 형식인 경우만 추출
            if re.match(r'^\d{2}\.\d{2}\.\d{2}$', raw_date):
                date = f"20{raw_date}"  # "20"을 추가하여 "YYYY.MM.DD" 형식으로 변경
            else:
                continue  # 날짜 형식이 맞지 않으면 스킵

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '노어노문학과',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "노어노문학과_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '노어노문학과_공지사항.csv'로 저장되었습니다.


In [ ]:
# 사학과

# 기본 URL 설정
base_url = "https://history.yonsei.ac.kr/histroy/02_announce/notice.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        tds = row.select('td')  # 모든 <td> 요소 선택

        # <td> 요소가 있는지 확인하고, 마지막 <td> 요소를 date로 추출
        if title_tag and tds:
            # 제목 텍스트를 불필요한 문자열을 제거하며 추출
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            raw_date = tds[-1].get_text(strip=True)  # 작성일 텍스트 추출

            # 날짜가 "YY.MM.DD" 형식인 경우만 추출
            if re.match(r'^\d{2}\.\d{2}\.\d{2}$', raw_date):
                date = f"20{raw_date}"  # "20"을 추가하여 "YYYY.MM.DD" 형식으로 변경
            else:
                continue  # 날짜 형식이 맞지 않으면 스킵

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '사학과',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "사학과_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '사학과_공지사항.csv'로 저장되었습니다.


In [ ]:
# 철학과

# 기본 URL 설정
base_url = "https://philosophy.yonsei.ac.kr/cholhak/process/process.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        tds = row.select('td')  # 모든 <td> 요소 선택

        # <td> 요소가 있는지 확인하고, 마지막 <td> 요소를 date로 추출
        if title_tag and tds:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            raw_date = tds[-1].get_text(strip=True)  # 작성일 텍스트 추출

            # 날짜가 "YY.MM.DD" 형식인 경우만 추출
            if re.match(r'^\d{2}\.\d{2}\.\d{2}$', raw_date):
                date = f"20{raw_date}"  # "20"을 추가하여 "YYYY.MM.DD" 형식으로 변경
            else:
                continue  # 날짜 형식이 맞지 않으면 스킵

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '철학과',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "철학과_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '철학과_공지사항.csv'로 저장되었습니다.


In [ ]:
# 문헌정보학과

# 기본 URL 설정
base_url = "https://yslis.yonsei.ac.kr/doai/board/notice.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        tds = row.select('td')  # 모든 <td> 요소 선택

        # <td> 요소가 있는지 확인하고, 마지막 <td> 요소를 date로 추출
        if title_tag and tds:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            raw_date = tds[-1].get_text(strip=True)  # 작성일 텍스트 추출

            # 날짜가 "YY.MM.DD" 형식인 경우만 추출
            if re.match(r'^\d{2}\.\d{2}\.\d{2}$', raw_date):
                date = f"20{raw_date}"  # "20"을 추가하여 "YYYY.MM.DD" 형식으로 변경
            else:
                continue  # 날짜 형식이 맞지 않으면 스킵

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '문헌정보학과',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "문헌정보학과_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '문헌정보학과_공지사항.csv'로 저장되었습니다.


In [ ]:
# 경제학과

import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import re
import urllib3

# SSL 경고 무시 설정
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# 기본 URL 설정
base_url = "https://econ.yonsei.ac.kr/econ/board/notice.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# 공지사항을 저장할 리스트
all_notices = []

def scrape_page(offset):
    """해당 페이지의 공지사항을 크롤링하는 함수"""
    params['article.offset'] = offset  # 페이지 오프셋 설정
    response = requests.get(base_url, params=params, verify=False)  # SSL 검증 비활성화
    response.raise_for_status()  # 요청 성공 확인

    soup = BeautifulSoup(response.text, 'html.parser')

    # 공지사항 목록에서 제목, 링크, 작성일 크롤링
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')  # 제목 추출
        tds = row.select('td')  # 모든 <td> 요소 선택

        # <td> 요소가 있는지 확인하고, 마지막 <td> 요소를 date로 추출
        if title_tag and tds:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])  # 절대 경로로 변환
            raw_date = tds[-1].get_text(strip=True)  # 작성일 텍스트 추출

            # 날짜가 "YY.MM.DD" 형식인 경우만 추출
            if re.match(r'^\d{2}\.\d{2}\.\d{2}$', raw_date):
                date = f"20{raw_date}"  # "20"을 추가하여 "YYYY.MM.DD" 형식으로 변경
            else:
                continue  # 날짜 형식이 맞지 않으면 스킵

            # 공지사항의 상세 페이지로 이동하여 내용 크롤링
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '경제학과',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """공지사항 상세 페이지에서 내용을 크롤링하는 함수"""
    response = requests.get(link, verify=False)  # SSL 검증 비활성화
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용이 담긴 <div class="fr-view">에서 텍스트 추출
    context_tag = soup.select_one('div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 1페이지부터 5페이지까지 크롤링
for page in range(5):
    offset = page * 10  # 각 페이지 오프셋은 10씩 증가
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# 데이터프레임 생성 후 제목과 작성일을 기준으로 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "경제학과_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '경제학과_공지사항.csv'로 저장되었습니다.


In [ ]:
# 응용통계학과

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Base URL
base_url = "https://stat.yonsei.ac.kr/stat/board/under_notice.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# List to store all notices
all_notices = []

def scrape_page(offset):
    """Scrapes a page of notices based on the offset."""
    params['article.offset'] = offset  # Set page offset
    response = requests.get(base_url, params=params, verify=False)  # Disable SSL verification
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all notices on the page
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')
        tds = row.select('td')

        if title_tag and tds:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])
            raw_date = tds[-1].get_text(strip=True)

            if re.match(r'^\d{2}\.\d{2}\.\d{2}$', raw_date):
                date = f"20{raw_date}"  # Convert to "YYYY.MM.DD"
            else:
                continue

            context = scrape_notice_context(link)

            all_notices.append({
                'department': '응용통계학과',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """Scrapes the content of an individual notice."""
    response = requests.get(link, verify=False)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    context_tag = soup.select_one('div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# Scraping the first 5 pages
for page in range(5):
    offset = page * 10
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# Remove duplicates based on title and date
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# Save to CSV
csv_filename = "응용통계학과_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '응용통계학과_공지사항.csv'로 저장되었습니다.


In [ ]:
# 수학과

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Base URL
base_url = "https://math.yonsei.ac.kr/math/math/notice.do"
params = {
    'mode': 'list',
    'articleLimit': 10
}

# List to store all notices
all_notices = []

def scrape_page(offset):
    """Scrapes a page of notices based on the offset."""
    params['article.offset'] = offset  # Set page offset
    response = requests.get(base_url, params=params, verify=False)  # Disable SSL verification
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all notices on the page
    for row in soup.select('tr'):
        title_tag = row.select_one('a.c-board-title')
        tds = row.select('td')

        if title_tag and tds:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])
            raw_date = tds[-1].get_text(strip=True)

            if re.match(r'^\d{2}\.\d{2}\.\d{2}$', raw_date):
                date = f"20{raw_date}"  # Convert to "YYYY.MM.DD"
            else:
                continue

            context = scrape_notice_context(link)

            all_notices.append({
                'department': '수학과',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """Scrapes the content of an individual notice."""
    response = requests.get(link, verify=False)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    context_tag = soup.select_one('div.fr-view')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# Scraping the first 5 pages
for page in range(5):
    offset = page * 10
    print(f"{page + 1}페이지 크롤링 중...")
    scrape_page(offset)

# Remove duplicates based on title and date
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# Save to CSV
csv_filename = "수학과_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '수학과_공지사항.csv'로 저장되었습니다.


In [47]:
# 물리학과

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Base URL
base_url = "https://physics.yonsei.ac.kr/notice/board"
params = {
    'page': 1
}

# List to store all notices
all_notices = []

def scrape_page(page):
    """Scrapes a page of notices based on the page number."""
    params['page'] = page  # Set page number
    response = requests.get(base_url, params=params, verify=False)  # Disable SSL verification
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all rows containing notices
    for row in soup.select('tr'):
        title_tag = row.select_one('td.td-subject a')
        date_tag = row.select_one('td.td-date')

        if title_tag and date_tag:
            title = title_tag.get_text(strip=True).replace("[공지]", "").strip()
            link = urljoin(base_url, title_tag['href'])
            date = date_tag.get_text(strip=True)

            # Get context from the detailed page
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '물리학과',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """Scrapes the content of an individual notice."""
    response = requests.get(link, verify=False)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract context from the notice content
    context_tag = soup.select_one('div.bw_contents.editor_contents')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# Scraping the first 5 pages
for page in range(1, 6):
    print(f"{page}페이지 크롤링 중...")
    scrape_page(page)

# Remove duplicates based on title and date
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# Check if the dataframe is not empty
if not df.empty:
    # Save to CSV
    csv_filename = "물리학과_공지사항.csv"
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")
else:
    print("데이터프레임이 비어 있습니다. HTML 구조를 확인해주세요.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '물리학과_공지사항.csv'로 저장되었습니다.


In [ ]:
# 화학과

# SSL 경고 비활성화
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Base URL
base_url = "https://chemyonsei.kr/board/notice"
params = {
    'p': 1
}

# 전체 공지사항 저장 리스트
all_notices = []

def scrape_page(page):
    """해당 페이지의 공지사항 목록을 크롤링합니다."""
    params['p'] = page  # 페이지 번호 설정
    response = requests.get(base_url, params=params, verify=False)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 공지사항 리스트에서 제목과 날짜를 찾습니다.
    for row in soup.select('a.text-small'):
        title_tag = row
        date_tag = row.find_next('time')
        
        if title_tag and date_tag:
            title = title_tag.get_text(strip=True)
            link = urljoin(base_url, title_tag['href'])
            date = date_tag.get_text(strip=True)

            # 공지사항의 상세 내용을 크롤링합니다.
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '화학과',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """개별 공지사항의 상세 내용을 크롤링합니다."""
    response = requests.get(link, verify=False)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용 크롤링
    context_tag = soup.select_one('div.editor-contents')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 첫 5페이지 크롤링
for page in range(1, 6):
    print(f"{page}페이지 크롤링 중...")
    scrape_page(page)

# 데이터프레임 생성 및 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "화학과_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
3페이지 크롤링 중...
4페이지 크롤링 중...
5페이지 크롤링 중...
CSV 파일이 '화학과_공지사항.csv'로 저장되었습니다.


In [4]:
# 지구시스템과학과

# SSL 경고 비활성화
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Base URL
base_url = "http://geo.yonsei.ac.kr/index.php"
params = {
    'hCode': 'BOARD',
    'bo_idx': 2,
    'page': 'list',
    'pg': 1
}

# 전체 공지사항 저장 리스트
all_notices = []

def scrape_page(page):
    """해당 페이지의 공지사항 목록을 크롤링합니다."""
    params['pg'] = page  # 페이지 번호 설정
    response = requests.get(base_url, params=params, verify=False)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 공지사항 리스트에서 제목과 날짜를 찾습니다.
    for row in soup.select('tr'):
        title_tag = row.select_one('a[href^="?page=view"]')
        date_tag = row.select_one('td:nth-last-child(1)')  # 날짜가 마지막 열에 있다고 가정
        
        if title_tag and date_tag:
            title = title_tag.get_text(strip=True)
            link = urljoin(base_url, title_tag['href'])
            date = date_tag.get_text(strip=True)

            # 공지사항의 상세 내용을 크롤링합니다.
            context = scrape_notice_context(link)

            all_notices.append({
                'department': '지구시스템과학과',
                'title': title,
                'date': date,
                'link': link,
                'context': context
            })

def scrape_notice_context(link):
    """개별 공지사항의 상세 내용을 크롤링합니다."""
    response = requests.get(link, verify=False)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 상세 내용 크롤링
    context_tag = soup.select_one('div.board_content')
    context = context_tag.get_text(strip=True) if context_tag else "내용 없음"
    
    return context

# 첫 5페이지 크롤링
for page in range(1, 3):
    print(f"{page}페이지 크롤링 중...")
    scrape_page(page)

# 데이터프레임 생성 및 중복 제거
df = pd.DataFrame(all_notices).drop_duplicates(subset=['title', 'date'])

# CSV 파일로 저장
csv_filename = "지구시스템과학과_공지사항.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 '{csv_filename}'로 저장되었습니다.")


1페이지 크롤링 중...
2페이지 크롤링 중...
CSV 파일이 '지구시스템과학과_공지사항.csv'로 저장되었습니다.
